In [ ]:
%pip install instagrapi Pillow python-dotenv

In [ ]:
import random
import time
import logging
from datetime import datetime
from instagrapi import Client
from instagrapi.exceptions import LoginRequired
import os
from dotenv import load_dotenv

# Setup logging
logger = logging.getLogger()
logging.basicConfig(level=logging.INFO)

# Initialize the Instagram Client
cl = Client()
# Add random delays between requests (2-5 seconds)
cl.delay_range = [2, 5]
# Set proxy (uncomment and replace with your SOAX credentials)
# cl.set_proxy("http://<api_key>:wifi;ca;;;toronto@proxy.soax.com:9137")

# Save credentials from user
USERNAME = os.getenv("INSTAGRAM_USERNAME")
PASSWORD = os.getenv("INSTAGRAM_PASSWORD")
TWOFA = input("Enter 2FA code: ")

def login():
    """
    Attempts to login to Instagram using either the provided session information
    or the provided username and password.
    """
    try:
        session = cl.load_settings("session.json")
        login_via_session = False
        login_via_pw = False

        if session:
            try:
                cl.set_settings(session)
                cl.login(USERNAME, PASSWORD, verification_code=TWOFA)

                # Check if session is valid
                try:
                    cl.get_timeline_feed()
                    logger.info("Successfully logged in via session")
                    login_via_session = True
                except LoginRequired:
                    logger.info("Session is invalid, need to login via username and password")
                    old_session = cl.get_settings()

                    # Use the same device uuids across logins
                    cl.set_settings({})
                    cl.set_uuids(old_session["uuids"])

                    cl.login(USERNAME, PASSWORD, verification_code=TWOFA)
                    login_via_pw = True
            except Exception as e:
                logger.info(f"Couldn't login user using session information: {e}")

        if not login_via_session and not login_via_pw:
            try:
                logger.info(f"Attempting to login via username and password. username: {USERNAME}")
                if cl.login(USERNAME, PASSWORD, verification_code=TWOFA):
                    login_via_pw = True
            except Exception as e:
                logger.info(f"Couldn't login user using username and password: {e}")

        if not login_via_pw and not login_via_session:
            raise Exception("Couldn't login user with either password or session")

        # Save session after successful login
        cl.dump_settings('session.json')
        return True

    except Exception as e:
        logger.error(f"Login failed: {e}")
        return False

# Follow random users
def follow_random_users(daily_limit=50):
    followed = 0
    followed_users = []  # Keep track of followed usernames

    # Fetch a pool of accounts to follow
    user_info = cl.user_info_by_username_v1(os.getenv("TARGET_INSTAGRAM_USERNAME")).model_dump()
    print(f"User info: {user_info}")
    user_id = user_info.get("pk")
    print(f"User ID: {user_id}")
    print("Fetching followers...")
    followers = cl.user_followers(user_id, amount=70)
    print(followers)

    if not followers:
        print("No followers found. Exiting...")
        return

    while followed < daily_limit:
        try:
            # Randomly select a user from the followers list
            user_id, user_data = random.choice(list(followers.items()))
            username = user_data.username

            # Follow the user
            cl.user_follow(user_id)
            print(f"Followed {username}")
            followed += 1
            followed_users.append(username)

            # Random delay between actions (2-10 seconds)
            delay = random.randint(1, 4)
            print(f"Waiting for {delay} seconds before the next action...")
            time.sleep(delay)

        except Exception as e:
            print(f"Error while following user: {e}")
            break
        # Log summary at the end
    print(f"\n=== Follow Session Summary ===")
    print(f"Total users followed: {followed}")
    print(f"Users followed: {', '.join(followed_users)}")
    print(f"============================\n")

# Main script
if __name__ == "__main__":
    login()
    follow_random_users()